## Google Drive のマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Github リポジトリのクローン

In [2]:
!git clone https://github.com/Fumiya74/ft_bmshj2018_coco_relu.git
%cd /content/ft_bmshj2018_coco_relu

Cloning into 'ft_bmshj2018_coco_relu'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 69 (delta 43), reused 44 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (69/69), 25.19 KiB | 12.60 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/ft_bmshj2018_coco_relu


### ※ランタイム中にリモートリポジトリの変更したときに実行

In [19]:
!git pull origin main

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 371 bytes | 371.00 KiB/s, done.
From https://github.com/Fumiya74/ft_bmshj2018_coco_relu
 * branch            main       -> FETCH_HEAD
   6328056..8792aec  main       -> origin/main
Updating 6328056..8792aec
Fast-forward
 src/train_finetune_relu.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


## データセットの準備
Google Drive やWebページからzipファイルをColabランタイム上にダウンロードして解凍する

In [3]:
import shutil
import zipfile
import os
from tqdm import tqdm

drive_train_dir = '/content/drive/MyDrive/datasets'
local_train_dir = '/content/data'

print(f"{local_train_dir}を作成します")
os.makedirs(local_train_dir, exist_ok=True)

# Colabにコピーして解凍
zip_filename = "coco224.zip"
zip_path_drive = os.path.join(drive_train_dir, zip_filename)
zip_path_local = os.path.join('/content', zip_filename)

# コピー
if not os.path.exists(zip_path_drive):
    print(f"❌ Drive上に {zip_filename} がありません。スキップします。")

print(f"{zip_filename}コピー中")
shutil.copy(zip_path_drive, zip_path_local)

# 解凍
print("解凍中")
with zipfile.ZipFile(zip_path_local, 'r') as zipf:
    zipf.extractall(local_train_dir)


print("✅ Colabランタイムに全て解凍されました")

/content/dataを作成します
coco224.zipコピー中
解凍中
✅ Colabランタイムに全て解凍されました


## 仮想環境セットアップ
これはやらなくてもいいことが多いが、Colabランタイムのライブラリとの依存関係が解決できない場合は行う。

In [4]:
# クリーンに作り直したい場合は次を有効化
!rm -rf /content/ftenv

# virtualenv で隔離環境を作成
%pip -q install -U virtualenv
!python -m virtualenv /content/ftenv

# venv の pip で必要物をインストール（CompressAIは numpy<2.0 必須）
!/content/ftenv/bin/pip -q install -U pip
!/content/ftenv/bin/pip -q install "numpy==1.26.4"
# T4 でまずは cu121（合わなければ後で cu124 に変更）
!/content/ftenv/bin/pip -q install torch==2.3.1 torchvision==0.18.1 --index-url https://download.pytorch.org/whl/cu121
!/content/ftenv/bin/pip -q install compressai==1.2.7 pytorch-msssim==1.0.0 wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 39.5 MB/s eta 0:00:00
created virtual environment CPython3.12.11.final.0-64 in 340ms
  creator CPython3Posix(dest=/content/ftenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.2
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


## AI学習ログ可視化ツールWandBの設定

In [ ]:
import wandb
wandb.login(key="YOUR_API_KEY")  # ブラウザ認可orAPIキー入力のどちらか
!WANDB_PROJECT=bmshj2018_relu wandb login

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: f-matsuzawa719 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Currently logged in as: f-matsuzawa719 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## 学習スクリプト実行

In [ ]:
%env PYTHONUNBUFFERED=1
!/content/ftenv/bin/python -m src.train_finetune_relu \
  --coco_dir /content/data \
  --use_prepared true \
  --quality 8 --epochs 10 --batch_size 16 --lr 3e-7 \
  --recon_every 1 --recon_count 16 --recon_dir /content/drive/MyDrive/ft_bmshj2018_coco_relu/recon/enc_only \
  --save_dir /content/drive/MyDrive/ft_bmshj2018_coco_relu/checkpoints/enc_only \
  --wandb true --replace_parts encoder --train_scope all \
  --amp_warmup_steps 15000 \
  --lr_warmup_steps 1000 \
  --max_grad_norm 1.0 \
  --overflow_check true

env: PYTHONUNBUFFERED=1
[wandb] Starting new run (id=ylc1h25s_encoder_all)
wandb: Currently logged in as: f-matsuzawa719 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ creating run (0.2s)
wandb: Tracking run with wandb version 0.21.3
wandb: Run data is saved locally in /content/ft_bmshj2018_coco_relu/wandb/run-20250906_094728-ylc1h25s_encoder_all
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ylc1h25s_encoder_all
wandb: ⭐️ View project at https://wandb.ai/f-matsuzawa719/bmshj2018_relu
wandb: 🚀 View run at https://wandb.ai/f-matsuzawa719/bmshj2018_relu/runs/ylc1h25s_encoder_all
[train] 118287 images found
[val] 5000 images found
[replace_gdn_with_relu] Replaced 3 GDN(s) -> ReLU (mode='encoder')
[recon] Saved 16 recon images to /content/drive/MyDrive/ft_bmshj2018_coco_relu/recon/enc_only/pre_e000
train e1/10:  86% 6369/7393 [35:25<05:41,  3.00it/s, loss=0.0878, msssim=0.9032, psnr=20.06, lr=3.00e-07,